<a href="https://colab.research.google.com/github/Aioshu/Browser/blob/master/CNN_ADE_VS_Non_ADE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, log_loss
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
#from imblearn.over_sampling import SMOTE
from collections import Counter
from scipy.sparse import hstack
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold 
from collections import Counter, defaultdict
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import normalized_mutual_info_score
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")
import os
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string
#from mlxtend.classifier import StackingClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
pwd

'/content'

In [ ]:
data_path = "/content/drive/MyDrive/AdverseDrugReaction-master/ADE-Corpus-V2/"
texts, labels,pubmed_ids = [], [],[]
with open(os.path.join(data_path, 'DRUG-AE.rel')) as f:
    for line in f:
        pubmed_id, text = line.strip().split('|')[:2]
        #print(pubmed_id)
        texts.append(text)
        pubmed_ids.append(pubmed_id)
        labels.append(1)

with open(os.path.join(data_path, 'ADE-NEG.txt')) as f:
    for i,line in enumerate(f):
        #print(line.strip().split(" ")[:2])
        if i<7000:
            pubmed_id, neg = line.strip().split(' ')[:2]
            text = ' '.join(line.strip().split(' ')[2:])
            texts.append(text)

            pubmed_ids.append(pubmed_id)
            labels.append(0)
        else:
            break
np.random.seed(1)
# Shuffle the data as Keras won't shuffle validation data.
# This can make the training ends early as we are using
# early stop for regularisation.
idx = np.random.permutation(len(labels))        
labels = np.asarray(labels)[idx]
pubmed_ida=np.asarray(pubmed_ids)[idx]
texts = np.asarray(texts, dtype='str')[idx]


In [ ]:
df={"Index":idx,"Pudmed_Id":pubmed_ida,"Text":texts,"Label":labels}#'Text', 'Label'
data=pd.DataFrame(df)

In [ ]:
data.head()

,Index,Pudmed_Id,Text,Label
0,6941,8623881,"CONCLUSIONS: Confocal microscopy can be a useful, noninvasive imaging technique helpful in the study, diagnosis, and treatment of Acanthamoeba keratitis.",0
1,12941,12209355,Successful treatment of post-transplant lymphoproliferative disorder in autologous blood stem cell transplant recipients.,0
2,5114,6810714,"The authors describe a woman with chronic schizophrenia who experienced delirium, grand mal seizure, and photosensitivity after the addition of propranolol to her neuroleptic regimen.",1
3,10028,12032026,Oral ketamine: a promising treatment for restless legs syndrome.,0
4,9398,10572611,"The results of an Intergroup trial, coordinated by the Southwest Oncology Group (SWOG) demonstrated markedly superior progression free and overall survival for combined chemoradiotherapy compared to radiotherapy alone.",0


In [ ]:
data.Label.value_counts()

0    7000
1    6821
Name: Label, dtype: int64

In [ ]:
pos = []
neg = []
for l in data.Label:
    if l == 0:
        pos.append(0)
        neg.append(1)
    elif l == 1:
        pos.append(1)
        neg.append(0)

data['Pos']= pos
data['Neg']= neg

In [ ]:

data.head()


,Index,Pudmed_Id,Text,Label,Pos,Neg
0,6941,8623881,"CONCLUSIONS: Confocal microscopy can be a useful, noninvasive imaging technique helpful in the study, diagnosis, and treatment of Acanthamoeba keratitis.",0,0,1
1,12941,12209355,Successful treatment of post-transplant lymphoproliferative disorder in autologous blood stem cell transplant recipients.,0,0,1
2,5114,6810714,"The authors describe a woman with chronic schizophrenia who experienced delirium, grand mal seizure, and photosensitivity after the addition of propranolol to her neuroleptic regimen.",1,1,0
3,10028,12032026,Oral ketamine: a promising treatment for restless legs syndrome.,0,0,1
4,9398,10572611,"The results of an Intergroup trial, coordinated by the Southwest Oncology Group (SWOG) demonstrated markedly superior progression free and overall survival for combined chemoradiotherapy compared to radiotherapy alone.",0,0,1


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct

data['Text_Clean'] = data['Text'].apply(lambda x: remove_punct(x))
from nltk import word_tokenize, WordNetLemmatizer
tokens = [word_tokenize(sen) for sen in data.Text_Clean]
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens]

In [ ]:
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

In [ ]:
def remove_stop_words(tokens): 
    return [word for word in tokens if word not in stoplist]

In [ ]:
filtered_words = [remove_stop_words(sen) for sen in lower_tokens]
result = [' '.join(sen) for sen in filtered_words]
data['Text_Final'] = result
data['tokens'] = filtered_words
data = data[['Text','Text_Final', 'tokens', 'Label', 'Pos', 'Neg']]


In [ ]:
data[:4]


,Text,Text_Final,tokens,Label,Pos,Neg
0,"CONCLUSIONS: Confocal microscopy can be a useful, noninvasive imaging technique helpful in the study, diagnosis, and treatment of Acanthamoeba keratitis.",conclusions confocal microscopy useful noninvasive imaging technique helpful study diagnosis treatment acanthamoeba keratitis,"[conclusions, confocal, microscopy, useful, noninvasive, imaging, technique, helpful, study, diagnosis, treatment, acanthamoeba, keratitis]",0,0,1
1,Successful treatment of post-transplant lymphoproliferative disorder in autologous blood stem cell transplant recipients.,successful treatment posttransplant lymphoproliferative disorder autologous blood stem cell transplant recipients,"[successful, treatment, posttransplant, lymphoproliferative, disorder, autologous, blood, stem, cell, transplant, recipients]",0,0,1
2,"The authors describe a woman with chronic schizophrenia who experienced delirium, grand mal seizure, and photosensitivity after the addition of propranolol to her neuroleptic regimen.",authors describe woman chronic schizophrenia experienced delirium grand mal seizure photosensitivity addition propranolol neuroleptic regimen,"[authors, describe, woman, chronic, schizophrenia, experienced, delirium, grand, mal, seizure, photosensitivity, addition, propranolol, neuroleptic, regimen]",1,1,0
3,Oral ketamine: a promising treatment for restless legs syndrome.,oral ketamine promising treatment restless legs syndrome,"[oral, ketamine, promising, treatment, restless, legs, syndrome]",0,0,1


# Split data into test and train

In [ ]:
data_train, data_test = train_test_split(data, 
                                         test_size=0.05, 
                                         random_state=42)

In [ ]:
pwd

'/content'

In [ ]:
data_train.to_csv("/content/drive/MyDrive/dataSummary/deep_model/data/data_train",sep="~")
data_test.to_csv("/content/drive/MyDrive/dataSummary/deep_model/data/data_test",sep="~")

In [ ]:
import pandas as pd
data_train=pd.read_csv("/content/drive/MyDrive/dataSummary/deep_model/data/data_train",sep="~")
data_test=pd.read_csv("/content/drive/MyDrive/dataSummary/deep_model/data/data_test",sep="~")

In [ ]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))


162871 words total, with a vocabulary size of 14502
Max sentence length is 58


In [ ]:
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

8601 words total, with a vocabulary size of 3407
Max sentence length is 36


In [ ]:
# word2vec_path = 'GoogleNews-vectors-negative300.bin.gz'
# word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [ ]:
#word2vec_path = 'GoogleNews-vectors-negative300.bin.gz'
#word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [ ]:
#from gensim.models.word2vec import Word2Vec
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
cd /content/drive/MyDrive/dataSummary/
!unzip glove*.zip

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

# Embedding

In [ ]:
word2vec=wv
import numpy as np

training_embeddings = get_word2vec_embeddings(word2vec, data_train, generate_missing=True)
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300


# Tokenize and Pad sequences¶


In [ ]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["Text_Final"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["Text_Final"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)


test_sequences = tokenizer.texts_to_sequences(data_test["Text_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)


Found 14502 unique tokens.
(14503, 300)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define CNN¶


In [ ]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [2,3,4,5,6]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)


    l_merge = concatenate(convs, axis=1)

    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# checkpoint
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    # simple early stopping
    #es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
    model.summary()
    return model


    

In [ ]:
len(x_train)

13129

In [ ]:
label_names = ['Pos', 'Neg']
y_train = data_train[label_names].values

x_train = train_cnn_data
y_tr = y_train

model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)))


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 300)      4350900     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 49, 200)      120200      embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 48, 200)      180200      embedding[0][0]                  
______________________________________________________________________________________________

# Train CNN¶


In [ ]:
mkdir /content/drive/MyDrive/dataSummary/deep_model/ashish

In [ ]:
num_epochs = 40
batch_size = 64
# checkpoint
# filepath="/content/drive/MyDrive/dataSummary/deep_model/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
filepath="/content/drive/MyDrive/dataSummary/deep_model/ashish/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
# Fit the model
# model.fit(X, Y, validation_split=0.33, epochs=150, batch_size=10, callbacks=callbacks_list, verbose=0)
hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.05, shuffle=True, batch_size=batch_size,callbacks=callbacks_list, verbose=0)




Epoch 00001: val_accuracy improved from -inf to 0.71135, saving model to /content/drive/MyDrive/dataSummary/deep_model/ashish/weights-improvement-01-0.71.hdf5

Epoch 00002: val_accuracy improved from 0.71135 to 0.71287, saving model to /content/drive/MyDrive/dataSummary/deep_model/ashish/weights-improvement-02-0.71.hdf5

Epoch 00003: val_accuracy did not improve from 0.71287

Epoch 00004: val_accuracy improved from 0.71287 to 0.71516, saving model to /content/drive/MyDrive/dataSummary/deep_model/ashish/weights-improvement-04-0.72.hdf5

Epoch 00005: val_accuracy did not improve from 0.71516

Epoch 00006: val_accuracy did not improve from 0.71516

Epoch 00007: val_accuracy did not improve from 0.71516

Epoch 00008: val_accuracy did not improve from 0.71516

Epoch 00009: val_accuracy did not improve from 0.71516

Epoch 00010: val_accuracy did not improve from 0.71516

Epoch 00011: val_accuracy improved from 0.71516 to 0.71896, saving model to /content/drive/MyDrive/dataSummary/deep_model

In [ ]:
from keras.models import model_from_json
path="/content/drive/MyDrive/dataSummary/deep_model/"
model_json = model.to_json()
with open(path+"model_new.json", "w") as json_file:
    json_file.write(model_json)
print("model_new")
# serialize weights to HDF5
# model.save_weights(path+"model_new.h5")
# print("Saved model to disk Hi")

model_new


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from keras.models import model_from_json
import numpy
# # create model
# model = Sequential()
# model.add(Dense(12, input_dim=8, activation='relu'))
# model.add(Dense(8, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# load json and create model
json_file = open('/content/drive/MyDrive/dataSummary/deep_model/model_new.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
#loaded_model.load_weights("/content/drive/MyDrive/dataSummary/deep_model/model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
#loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# load weights
loaded_model.load_weights("/content/drive/MyDrive/dataSummary/deep_model/weights-improvement-36-0.88.hdf5")
# Compile model (required to make predictions)
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# print("Created model and loaded weights from file")
# # load pima indians dataset
# dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# # split into input (X) and output (Y) variables
# X = dataset[:,0:8]
# Y = dataset[:,8]
# # estimate accuracy on whole dataset using loaded weights
# scores = model.evaluate(X, Y, verbose=0)
# print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Loaded model from disk


In [ ]:
list(loaded_model.count_params)

TypeError: ignored

## Evalution of Model 

In [ ]:
# Plot the loss and accuracy curves for training and validation 
plt.plot(loaded_model.history['val_loss'], color='b', label="validation loss")
plt.title("Test Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

TypeError: ignored

In [ ]:
predictions = loaded_model.predict(test_cnn_data, batch_size=1024, verbose=1)
labels = [1, 0]


prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])




1/1 [==============================] - 0s 358ms/step


In [ ]:
sum(data_test.Label==prediction_labels)/len(prediction_labels)



0.8916184971098265

In [ ]:
len(prediction_labels)

692

In [ ]:
from sklearn.metrics import precision_score,accuracy_score,f1_score,recall_score

## Picking the vectorizer
print(precision_score(prediction_labels, data_test.Label))
print(recall_score(prediction_labels, data_test.Label))
print(f1_score(prediction_labels, data_test.Label))


0.869942196531792
0.9093655589123867
0.8892171344165437


In [ ]:
from sklearn.metrics import precision_score,accuracy_score,f1_score,recall_score,classification_report
print(precision_score(prediction_labels, data_test.Label))
print(recall_score(prediction_labels, data_test.Label))
print(f1_score(prediction_labels, data_test.Label))
print(classification_report(prediction_labels, data_test.Label))

0.869942196531792
0.9093655589123867
0.8892171344165437
              precision    recall  f1-score   support

           0       0.91      0.88      0.89       361
           1       0.87      0.91      0.89       331

    accuracy                           0.89       692
   macro avg       0.89      0.89      0.89       692
weighted avg       0.89      0.89      0.89       692



In [ ]:
from sklearn.metrics import precision_score,accuracy_score,f1_score,recall_score,classification_report
print(precision_score(prediction_labels, data_test.Label))
print(recall_score(prediction_labels, data_test.Label))
print(f1_score(prediction_labels, data_test.Label))
print(classification_report(prediction_labels, data_test.Label))

0.869942196531792
0.9093655589123867
0.8892171344165437
              precision    recall  f1-score   support

           0       0.91      0.88      0.89       361
           1       0.87      0.91      0.89       331

    accuracy                           0.89       692
   macro avg       0.89      0.89      0.89       692
weighted avg       0.89      0.89      0.89       692



In [ ]:
data_test.Label.value_counts()


1    346
0    346
Name: Label, dtype: int64

In [ ]:
data_test

,Text,Text_Final,tokens,Label,Pos,Neg
4288,In-vitro rechallenge of the patient's lymphocytes with cytochrome P-450 generated metabolites of phenobarbital showed extensive cytotoxicity compared to control.,invitro rechallenge patients lymphocytes cytochrome p450 generated metabolites phenobarbital showed extensive cytotoxicity compared control,"[invitro, rechallenge, patients, lymphocytes, cytochrome, p450, generated, metabolites, phenobarbital, showed, extensive, cytotoxicity, compared, control]",0,0,1
8196,"PURPOSE: The case of a patient who developed aseptic meningitis, hemolytic anemia, hepatitis, and orthostatic hypotension simultaneously during treatment with trimethoprim-sulfamethoxazole is described.",purpose case patient developed aseptic meningitis hemolytic anemia hepatitis orthostatic hypotension simultaneously treatment trimethoprimsulfamethoxazole described,"[purpose, case, patient, developed, aseptic, meningitis, hemolytic, anemia, hepatitis, orthostatic, hypotension, simultaneously, treatment, trimethoprimsulfamethoxazole, described]",1,1,0
7558,The necessity of placing such grafts behind the ureters is reiterated and the routine performance of preoperative and postoperative excretory urography is suggested.,necessity placing grafts behind ureters reiterated routine performance preoperative postoperative excretory urography suggested,"[necessity, placing, grafts, behind, ureters, reiterated, routine, performance, preoperative, postoperative, excretory, urography, suggested]",0,0,1
7198,Retinal dysfunction and anterior segment deposits in a patient treated with rifabutin.,retinal dysfunction anterior segment deposits patient treated rifabutin,"[retinal, dysfunction, anterior, segment, deposits, patient, treated, rifabutin]",1,1,0
4639,An 83-year-old man receiving glipizide 10 mg bid developed symptomatic hypoglycemia within three days of adding trimethoprim/sulfamethoxazole (TMP/SMX) to his regimen.,83yearold man receiving glipizide 10 mg bid developed symptomatic hypoglycemia within three days adding trimethoprimsulfamethoxazole tmpsmx regimen,"[83yearold, man, receiving, glipizide, 10, mg, bid, developed, symptomatic, hypoglycemia, within, three, days, adding, trimethoprimsulfamethoxazole, tmpsmx, regimen]",1,1,0
...,...,...,...,...,...,...
1530,Clinic records were reviewed to extract clinical and laboratory data.,clinic records reviewed extract clinical laboratory data,"[clinic, records, reviewed, extract, clinical, laboratory, data]",0,0,1
12398,Systemic allergic reactions to aprotinin injection around the Achilles tendon.,systemic allergic reactions aprotinin injection around achilles tendon,"[systemic, allergic, reactions, aprotinin, injection, around, achilles, tendon]",0,0,1
10187,"3. Thus, the decreased plasma cortisol level during alprazolam treatment of panic disorder was suggested to be caused not by symptom alleviation due to alprazolam but by alprazolam administration itself.",3 thus decreased plasma cortisol level alprazolam treatment panic disorder suggested caused symptom alleviation due alprazolam alprazolam administration,"[3, thus, decreased, plasma, cortisol, level, alprazolam, treatment, panic, disorder, suggested, caused, symptom, alleviation, due, alprazolam, alprazolam, administration]",1,1,0
3366,PURPOSE: To report two cases of acute endophthalmitis following intravitreal bevacizumab injection.,purpose report two cases acute endophthalmitis following intravitreal bevacizumab injection,"[purpose, report, two, cases, acute, endophthalmitis, following, intravitreal, bevacizumab, injection]",1,1,0


In [ ]:
# serialize model to JSON
# later...
from keras.models import model_from_json
path="/content/drive/MyDrive/dataSummary/deep_model/"
model_json = model.to_json()
with open(path+"model_new.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(path+"model_new.h5")
print("Saved model to disk Hi")

NameError: ignored

In [ ]:

# load json and create model
json_file = open('/content/drive/MyDrive/dataSummary/deep_model/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/drive/MyDrive/dataSummary/deep_model/model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


Loaded model from disk


In [ ]:
loaded_model

In [ ]:
score = loaded_model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

NameError: ignored

In [ ]:
data_test["prediction_labels"]=prediction_labels

In [ ]:
data_test

,Text,Text_Final,tokens,Label,Pos,Neg,prediction_labels
4288,In-vitro rechallenge of the patient's lymphocytes with cytochrome P-450 generated metabolites of phenobarbital showed extensive cytotoxicity compared to control.,invitro rechallenge patients lymphocytes cytochrome p450 generated metabolites phenobarbital showed extensive cytotoxicity compared control,"[invitro, rechallenge, patients, lymphocytes, cytochrome, p450, generated, metabolites, phenobarbital, showed, extensive, cytotoxicity, compared, control]",0,0,1,0
8196,"PURPOSE: The case of a patient who developed aseptic meningitis, hemolytic anemia, hepatitis, and orthostatic hypotension simultaneously during treatment with trimethoprim-sulfamethoxazole is described.",purpose case patient developed aseptic meningitis hemolytic anemia hepatitis orthostatic hypotension simultaneously treatment trimethoprimsulfamethoxazole described,"[purpose, case, patient, developed, aseptic, meningitis, hemolytic, anemia, hepatitis, orthostatic, hypotension, simultaneously, treatment, trimethoprimsulfamethoxazole, described]",1,1,0,1
7558,The necessity of placing such grafts behind the ureters is reiterated and the routine performance of preoperative and postoperative excretory urography is suggested.,necessity placing grafts behind ureters reiterated routine performance preoperative postoperative excretory urography suggested,"[necessity, placing, grafts, behind, ureters, reiterated, routine, performance, preoperative, postoperative, excretory, urography, suggested]",0,0,1,0
7198,Retinal dysfunction and anterior segment deposits in a patient treated with rifabutin.,retinal dysfunction anterior segment deposits patient treated rifabutin,"[retinal, dysfunction, anterior, segment, deposits, patient, treated, rifabutin]",1,1,0,1
4639,An 83-year-old man receiving glipizide 10 mg bid developed symptomatic hypoglycemia within three days of adding trimethoprim/sulfamethoxazole (TMP/SMX) to his regimen.,83yearold man receiving glipizide 10 mg bid developed symptomatic hypoglycemia within three days adding trimethoprimsulfamethoxazole tmpsmx regimen,"[83yearold, man, receiving, glipizide, 10, mg, bid, developed, symptomatic, hypoglycemia, within, three, days, adding, trimethoprimsulfamethoxazole, tmpsmx, regimen]",1,1,0,1
...,...,...,...,...,...,...,...
1530,Clinic records were reviewed to extract clinical and laboratory data.,clinic records reviewed extract clinical laboratory data,"[clinic, records, reviewed, extract, clinical, laboratory, data]",0,0,1,0
12398,Systemic allergic reactions to aprotinin injection around the Achilles tendon.,systemic allergic reactions aprotinin injection around achilles tendon,"[systemic, allergic, reactions, aprotinin, injection, around, achilles, tendon]",0,0,1,1
10187,"3. Thus, the decreased plasma cortisol level during alprazolam treatment of panic disorder was suggested to be caused not by symptom alleviation due to alprazolam but by alprazolam administration itself.",3 thus decreased plasma cortisol level alprazolam treatment panic disorder suggested caused symptom alleviation due alprazolam alprazolam administration,"[3, thus, decreased, plasma, cortisol, level, alprazolam, treatment, panic, disorder, suggested, caused, symptom, alleviation, due, alprazolam, alprazolam, administration]",1,1,0,1
3366,PURPOSE: To report two cases of acute endophthalmitis following intravitreal bevacizumab injection.,purpose report two cases acute endophthalmitis following intravitreal bevacizumab injection,"[purpose, report, two, cases, acute, endophthalmitis, following, intravitreal, bevacizumab, injection]",1,1,0,1


In [ ]:
pd.set_option("max_colwidth",1000)
data_test[['Text','Text_Final', 'tokens', 'Label', 'prediction_labels']][data_test["Label"]!=data_test["prediction_labels"]].sample(10)

,Text,Text_Final,tokens,Label,prediction_labels
3837,Case 3: A 29-year-old female alcoholic complained of general fatigue and a slight fever after 1.5 years of abstinence with cyanamide treatment.,case 3 29yearold female alcoholic complained general fatigue slight fever 15 years abstinence cyanamide treatment,"[case, 3, 29yearold, female, alcoholic, complained, general, fatigue, slight, fever, 15, years, abstinence, cyanamide, treatment]",1,0
6192,We report the case of a patient who co-ingested a tricyclic antidepressant (2500 mg of doxepin) and a neuroleptic drug (3500 mg of prothipendyl).,report case patient coingested tricyclic antidepressant 2500 mg doxepin neuroleptic drug 3500 mg prothipendyl,"[report, case, patient, coingested, tricyclic, antidepressant, 2500, mg, doxepin, neuroleptic, drug, 3500, mg, prothipendyl]",0,1
31,"In four patients, spasm occurred spontaneous and in one patient after 0.05 mg of ergonovine.",four patients spasm occurred spontaneous one patient 005 mg ergonovine,"[four, patients, spasm, occurred, spontaneous, one, patient, 005, mg, ergonovine]",1,0
357,"Benzocaine-induced methemoglobinemia has been reported in man, dogs, and cats.",benzocaineinduced methemoglobinemia reported man dogs cats,"[benzocaineinduced, methemoglobinemia, reported, man, dogs, cats]",1,0
2157,Transient visual anomalies associated with drug treatment for spastic colon.,transient visual anomalies associated drug treatment spastic colon,"[transient, visual, anomalies, associated, drug, treatment, spastic, colon]",0,1
9957,Unusual cause of methadone poisoning.,unusual cause methadone poisoning,"[unusual, cause, methadone, poisoning]",0,1
6418,A patient with generalized MG was effectively managed with MM but developed CNS lymphoma after 3 years of treatment.,patient generalized mg effectively managed mm developed cns lymphoma 3 years treatment,"[patient, generalized, mg, effectively, managed, mm, developed, cns, lymphoma, 3, years, treatment]",1,0
6561,The elderly are also particularly prone to develop toxic side effects from standard antidepressive agents.,elderly also particularly prone develop toxic side effects standard antidepressive agents,"[elderly, also, particularly, prone, develop, toxic, side, effects, standard, antidepressive, agents]",0,1
9835,Case 3: A 29-year-old female alcoholic complained of general fatigue and a slight fever after 1.5 years of abstinence with cyanamide treatment.,case 3 29yearold female alcoholic complained general fatigue slight fever 15 years abstinence cyanamide treatment,"[case, 3, 29yearold, female, alcoholic, complained, general, fatigue, slight, fever, 15, years, abstinence, cyanamide, treatment]",1,0
10999,"Some days after the administration of a third bolus of ABVD (adriamycin, bleomycin, vinblastine, dacarbazine) a patient affected by immunoblastic lymphoma underwent a neurotoxic crisis.",days administration third bolus abvd adriamycin bleomycin vinblastine dacarbazine patient affected immunoblastic lymphoma underwent neurotoxic crisis,"[days, administration, third, bolus, abvd, adriamycin, bleomycin, vinblastine, dacarbazine, patient, affected, immunoblastic, lymphoma, underwent, neurotoxic, crisis]",0,1
